### CRM Sales Opportunities Analysis - Data Integration

**Business Context:** Multi-table CRM database integration<br>
**Objective:** Merge preprocessed tables into analysis-ready master dataset<br>
**Expected Outcome:** Clean integrated dataset ready for EDA and analysis



#### Step 1: Import libraries and setup

In [21]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
from pathlib import Path
import warnings
warnings.filterwarnings('ignore')

# Configure plotting
plt.style.use('seaborn-v0_8')
sns.set_palette("husl")
plt.rcParams['figure.figsize'] = (12, 8)

print("CRM DATA INTEGRATION PIPELINE")
print("=" * 60)
print("Combining preprocessed tables into master dataset")
print("=" * 60)

CRM DATA INTEGRATION PIPELINE
Combining preprocessed tables into master dataset


#### Step 2: Load preprocessing tables

In [22]:
print("\nSTEP 2: LOADING PREPROCESSED TABLES")
print("-" * 50)

# Load all preprocessed tables
data_path = Path('../data/interim/')
tables = {}

table_files = {
    'accounts': 'accounts_preprocessed.csv',
    'products': 'products_preprocessed.csv', 
    'sales_pipeline': 'sales_pipeline_preprocessed.csv',
    'sales_teams': 'sales_teams_preprocessed.csv',
    'data_dictionary': 'data_dictionary_preprocessed.csv'
}

for table_name, filename in table_files.items():
    file_path = data_path / filename
    try:
        df = pd.read_csv(file_path)
        tables[table_name] = df
        print(f"Loaded {table_name}: {df.shape[0]:,} rows X {df.shape[1]} columns")
        
        # Convert datetime columns back to datetime 
        if table_name == 'sales_pipeline':
            df['engage_date'] = pd.to_datetime(df['engage_date'])
            df['close_date'] = pd.to_datetime(df['close_date'])
            
    except FileNotFoundError:
        print(f"Error: {filename} not found")
    except Exception as e:
        print(f"Error loading {filename}: {str(e)}")

print(f"\nSuccessfully loaded {len(tables)} tables for integration")



STEP 2: LOADING PREPROCESSED TABLES
--------------------------------------------------
Loaded accounts: 85 rows X 7 columns
Loaded products: 7 rows X 3 columns
Loaded sales_pipeline: 8,800 rows X 8 columns
Loaded sales_teams: 35 rows X 3 columns
Loaded data_dictionary: 21 rows X 3 columns

Successfully loaded 5 tables for integration


#### Step 3: Examine table structures and their relationships

In [23]:
print("\nSTEP 3: TABLE STRUCTURE ANALYSIS")
print("-" * 50)

for table_name, df in tables.items():
    print(f"\n{table_name.upper()}:")
    print(f"Shape: {df.shape[0]:,} rows X {df.shape[1]} columns")
    print(f"Columns: {list(df.columns)}")
    
    # Show key columns for relationship identification
    if table_name == 'sales_pipeline':
        print(f"  Key relationships: account, product, sales_agent")
    elif table_name == 'accounts':
        print(f"  Primary key: account")
    elif table_name == 'products':
        print(f"  Primary key: product")
    elif table_name == 'sales_teams':
        print(f"  Primary key: sales_agent")



STEP 3: TABLE STRUCTURE ANALYSIS
--------------------------------------------------

ACCOUNTS:
Shape: 85 rows X 7 columns
Columns: ['account', 'sector', 'year_established', 'revenue', 'employees', 'office_location', 'subsidiary_of']
  Primary key: account

PRODUCTS:
Shape: 7 rows X 3 columns
Columns: ['product', 'series', 'sales_price']
  Primary key: product

SALES_PIPELINE:
Shape: 8,800 rows X 8 columns
Columns: ['opportunity_id', 'sales_agent', 'product', 'account', 'deal_stage', 'engage_date', 'close_date', 'close_value']
  Key relationships: account, product, sales_agent

SALES_TEAMS:
Shape: 35 rows X 3 columns
Columns: ['sales_agent', 'manager', 'regional_office']
  Primary key: sales_agent

DATA_DICTIONARY:
Shape: 21 rows X 3 columns
Columns: ['table', 'field', 'description']


#### Step 4: Validate foreign key relationships

In [24]:
print("\nSTEP 4: RELATIONSHIP VALIDATION")
print("-" * 50)

def validate_foreign_keys(primary_table, foreign_table, primary_key, foreign_key, relationship_name):
    """Validate foreign key relationships between tables"""
    
    # Get unique values from both tables
    primary_values = set(primary_table[primary_key].dropna())
    foreign_values = set(foreign_table[foreign_key].dropna())
    
    # Calculate match statistics
    total_foreign = len(foreign_values)
    matched = len(foreign_values.intersection(primary_values))
    unmatched = len(foreign_values - primary_values)
    match_rate = (matched / total_foreign * 100) if total_foreign > 0 else 0
    
    print(f"\n{relationship_name}:")
    print(f"Foreign key values: {total_foreign:,}")
    print(f"Matched: {matched:,} ({match_rate:.1f}%)")
    print(f"Unmatched: {unmatched:,}")
    
    if unmatched > 0:
        unmatched_values = list(foreign_values - primary_values)[:5]  # Show first 5
        print(f"  Sample unmatched: {unmatched_values}")
    
    return {
        'relationship': relationship_name,
        'total_foreign': total_foreign,
        'matched': matched,
        'unmatched': unmatched,
        'match_rate': match_rate
    }

# Validate all key relationships
validation_results = []

# sales_pipeline → accounts
result1 = validate_foreign_keys(
    tables['accounts'], tables['sales_pipeline'], 
    'account', 'account', 
    "Sales Pipeline → Accounts"
)
validation_results.append(result1)

# sales_pipeline → products  
result2 = validate_foreign_keys(
    tables['products'], tables['sales_pipeline'],
    'product', 'product',
    "Sales Pipeline → Products"
)
validation_results.append(result2)

# sales_pipeline → sales_teams
result3 = validate_foreign_keys(
    tables['sales_teams'], tables['sales_pipeline'],
    'sales_agent', 'sales_agent', 
    "Sales Pipeline → Sales Teams"
)
validation_results.append(result3)



STEP 4: RELATIONSHIP VALIDATION
--------------------------------------------------

Sales Pipeline → Accounts:
Foreign key values: 85
Matched: 85 (100.0%)
Unmatched: 0

Sales Pipeline → Products:
Foreign key values: 7
Matched: 6 (85.7%)
Unmatched: 1
  Sample unmatched: ['GTXPro']

Sales Pipeline → Sales Teams:
Foreign key values: 30
Matched: 30 (100.0%)
Unmatched: 0


#### Step 5: Create integrated master dataset

In [25]:
print("\nSTEP 5: MASTER DATASET INTEGRATION")
print("-" * 50)

# Start with sales_pipeline as the primary table (contains all opportunities)
master_df = tables['sales_pipeline'].copy()
print(f"Starting with sales_pipeline: {master_df.shape[0]:,} rows")

# Left join with accounts (company information)
print("Joining with accounts table...")
master_df = master_df.merge(
    tables['accounts'], 
    on='account', 
    how='left', 
    suffixes=('', '_account')
)
print(f"After joining accounts: {master_df.shape[0]:,} rows × {master_df.shape[1]} columns")

# Left join with products (product details and pricing)
print("Joining with products table...")
master_df = master_df.merge(
    tables['products'],
    on='product',
    how='left',
    suffixes=('', '_product')
)
print(f"After joining products: {master_df.shape[0]:,} rows × {master_df.shape[1]} columns")

# Left join with sales_teams (sales agent details)
print("Joining with sales_teams table...")
master_df = master_df.merge(
    tables['sales_teams'],
    on='sales_agent', 
    how='left',
    suffixes=('', '_team')
)
print(f"After joining sales_teams: {master_df.shape[0]:,} rows × {master_df.shape[1]} columns")

print(f"\nFinal integrated dataset: {master_df.shape[0]:,} rows × {master_df.shape[1]} columns")



STEP 5: MASTER DATASET INTEGRATION
--------------------------------------------------
Starting with sales_pipeline: 8,800 rows
Joining with accounts table...
After joining accounts: 8,800 rows × 14 columns
Joining with products table...
After joining products: 8,800 rows × 16 columns
Joining with sales_teams table...
After joining sales_teams: 8,800 rows × 18 columns

Final integrated dataset: 8,800 rows × 18 columns


#### Step 6: Data quality assessment for integrated dataset

In [26]:
print("\nSTEP 6: INTEGRATED DATASET QUALITY ASSESSMENT")
print("-" * 50)

# Missing value analysis
missing_summary = master_df.isnull().sum()
missing_pct = (missing_summary / len(master_df) * 100).round(1)

print("Missing values in integrated dataset:")
missing_df = pd.DataFrame({
    'Column': missing_summary.index,
    'Missing_Count': missing_summary.values,
    'Missing_Pct': missing_pct.values
}).query('Missing_Count > 0').sort_values('Missing_Count', ascending=False)

if len(missing_df) > 0:
    for _, row in missing_df.head(10).iterrows():
        print(f"{row['Column']}: {row['Missing_Count']:,} ({row['Missing_Pct']:.1f}%)")
else:
    print("  No missing values detected!")

# Data completeness by key business dimensions
print(f"\nData completeness analysis:")
print(f"Total opportunities: {len(master_df):,}")

# Check coverage from each joined table
if 'revenue' in master_df.columns:
    account_coverage = master_df['revenue'].notna().sum()
    print(f"Opportunities with account data: {account_coverage:,} ({account_coverage/len(master_df)*100:.1f}%)")

if 'sales_price' in master_df.columns:
    product_coverage = master_df['sales_price'].notna().sum()
    print(f"Opportunities with product data: {product_coverage:,} ({product_coverage/len(master_df)*100:.1f}%)")

if 'manager' in master_df.columns:
    team_coverage = master_df['manager'].notna().sum()
    print(f"Opportunities with sales team data: {team_coverage:,} ({team_coverage/len(master_df)*100:.1f}%)")



STEP 6: INTEGRATED DATASET QUALITY ASSESSMENT
--------------------------------------------------
Missing values in integrated dataset:
series: 1,480 (16.8%)
sales_price: 1,480 (16.8%)
close_value: 500 (5.7%)

Data completeness analysis:
Total opportunities: 8,800
Opportunities with account data: 8,800 (100.0%)
Opportunities with product data: 7,320 (83.2%)
Opportunities with sales team data: 8,800 (100.0%)


#### Step 7: Integration validation

In [27]:
print("\nSTEP 7: INTEGRATION VALIDATION")
print("-" * 50)

# Validate data integrity
print("Data integrity checks:")

# Check for unexpected duplicates
duplicate_opps = master_df['opportunity_id'].duplicated().sum()
print(f"Duplicate opportunities: {duplicate_opps} (should be 0)")

# Check join effectiveness  
total_ops = len(master_df)

if 'revenue' in master_df.columns:
    account_join_success = (master_df['revenue'].notna().sum() / total_ops * 100)
    print(f"Account data joined: {account_join_success:.1f}% of opportunities")

if 'sales_price' in master_df.columns:
    product_join_success = (master_df['sales_price'].notna().sum() / total_ops * 100) 
    print(f"Product data joined: {product_join_success:.1f}% of opportunities")

if 'manager' in master_df.columns:
    team_join_success = (master_df['manager'].notna().sum() / total_ops * 100)
    print(f"Team data joined: {team_join_success:.1f}% of opportunities")

# Business logic validation
if 'deal_stage' in master_df.columns and 'close_value' in master_df.columns:
    won_deals_with_value = master_df[
        (master_df['deal_stage'] == 'Won') & 
        (master_df['close_value'].notna())
    ].shape[0]
    total_won_deals = (master_df['deal_stage'] == 'Won').sum()
    
    if total_won_deals > 0:
        print(f"Won deals with values: {won_deals_with_value}/{total_won_deals} ({won_deals_with_value/total_won_deals*100:.1f}%)")



STEP 7: INTEGRATION VALIDATION
--------------------------------------------------
Data integrity checks:
Duplicate opportunities: 0 (should be 0)
Account data joined: 100.0% of opportunities
Product data joined: 83.2% of opportunities
Team data joined: 100.0% of opportunities
Won deals with values: 4238/4238 (100.0%)


#### Step 8: Display a sample of integrated data

In [28]:
print("\nSTEP 8: INTEGRATED DATASET PREVIEW")
print("-" * 50)

print("Sample of integrated data:")
print(master_df.head())

print(f"\nDataset info:")
print(f"Shape: {master_df.shape}")

print(f"\nColumn summary:")
for col in master_df.columns:
    non_null = master_df[col].notna().sum()
    data_type = master_df[col].dtype
    print(f"{col}: {non_null:,}/{len(master_df):,} non-null ({data_type})")



STEP 8: INTEGRATED DATASET PREVIEW
--------------------------------------------------
Sample of integrated data:
  opportunity_id      sales_agent         product  account deal_stage  \
0       1C1I7A6R      Moses Frase  GTX Plus Basic  Cancity        Won   
1       Z063OYW0  Darcel Schlecht          GTXPro    Isdom        Won   
2       EC4QE1BX  Darcel Schlecht      MG Special  Cancity        Won   
3       MV1LWRNH      Moses Frase       GTX Basic  Codehow        Won   
4       PE84CX4O        Zane Levy       GTX Basic   Hatfan        Won   

  engage_date close_date  close_value    sector  year_established  revenue  \
0  2016-10-20 2017-03-01       1054.0    retail              2001   718.62   
1  2016-10-25 2017-03-11       4514.0   medical              2002  3178.24   
2  2016-10-25 2017-03-07         50.0    retail              2001   718.62   
3  2016-10-25 2017-03-09        588.0  software              1998  2714.90   
4  2016-10-25 2017-03-02        517.0  services          

#### Step 9: Save the integrated dataset

In [29]:
print("\nSTEP 8: SAVING INTEGRATED DATASET")
print("-" * 50)

# Create processed data directory
output_path = Path('../data/processed/')
output_path.mkdir(parents=True, exist_ok=True)

# Save master integrated dataset
master_filename = 'crm_master_dataset.csv'
master_filepath = output_path / master_filename
master_df.to_csv(master_filepath, index=False)
print(f"Saved {master_filename}: {master_df.shape[0]:,} rows X {master_df.shape[1]} columns")

# Save integration summary
integration_summary = {
    'metric': [
        'Total Opportunities',
        'Total Features',
        'Original Tables Integrated',
        'Account Data Coverage',
        'Product Data Coverage', 
        'Sales Team Data Coverage',
        'Complete Records'
    ],
    'value': [
        len(master_df),
        master_df.shape[1],
        len(tables) - 1,  
        master_df['revenue'].notna().sum() if 'revenue' in master_df.columns else 0,
        master_df['sales_price'].notna().sum() if 'sales_price' in master_df.columns else 0,
        master_df['manager'].notna().sum() if 'manager' in master_df.columns else 0,
        master_df.dropna().shape[0]
    ]
}

summary_df = pd.DataFrame(integration_summary)
summary_df.to_csv(output_path / 'integration_summary.csv', index=False)
print(f"Saved integration_summary.csv")

print(f"\n" + "=" * 60)
print("DATA INTEGRATION COMPLETED")
print("=" * 60)
print(f"Master dataset: {master_df.shape[0]:,} opportunities × {master_df.shape[1]} features")
print(f"Integration success rates:")
for result in validation_results:
    print(f"{result['relationship']}: {result['match_rate']:.1f}%")
print(f"Output location: {output_path}")
print("Ready for exploratory data analysis!")


STEP 8: SAVING INTEGRATED DATASET
--------------------------------------------------
Saved crm_master_dataset.csv: 8,800 rows X 18 columns
Saved integration_summary.csv

DATA INTEGRATION COMPLETED
Master dataset: 8,800 opportunities × 18 features
Integration success rates:
Sales Pipeline → Accounts: 100.0%
Sales Pipeline → Products: 85.7%
Sales Pipeline → Sales Teams: 100.0%
Output location: ../data/processed
Ready for exploratory data analysis!
